## Calibration

The `Calibration` class provides a way to adjust weights of observations in a dataset to match specified target values. This is commonly used in survey research and policy modeling for rebalancing datasets to better represent desired population characteristics. 

The calibration process uses an optimization algorithm to find weights that minimize the distance from the original weights while achieving the target constraints.

## Basic usage

### Parameters

`__init__(data, weights, targets)`

- `data` (pd.DataFrame): The dataset to be calibrated. This should contain all the variables you want to use for calibration.
- `weights` (np.ndarray): Initial weights for each observation in the dataset. Typically starts as an array of ones for equal weighting.
- `targets` (np.ndarray): Target values that the calibration process should achieve. These correspond to the desired weighted sums.

Calibration can be easily done by initializing the `Calibration` class, passing in the parameters above. Then `calibrate()` method performs the actual calibration using the reweight function. This method:
- Adjusts the weights to better match the target values
- May subsample the data for efficiency
- Updates both `self.weights` and `self.data` with the calibrated results

## Example

Below is a complete example showing how to calibrate a dataset to match income targets for specific age groups:

In [2]:
from microcalibrate.calibration import Calibration
import numpy as np
import pandas as pd

# Create a sample dataset with age and income data
data = pd.DataFrame({
    "age": np.random.randint(18, 70, size=100),
    "income": np.random.normal(40000, 50000, size=100),
})

# Set initial weights (all equal in this example)
weights = np.ones(len(data))

# Calculate target values: total income for age groups 20-30 and 40-50 (as an example) or employ existing targets
targets_matrix = pd.DataFrame({
    "income_aged_20_30": ((data["age"] >= 20) & (data["age"] <= 30)).astype(float) * data["income"],
    "income_aged_40_50": ((data["age"] >= 40) & (data["age"] <= 50)).astype(float) * data["income"],
})

targets = np.array([
    targets_matrix["income_aged_20_30"].sum() * 1.10,  # 10% increase for 20-30 age group
    targets_matrix["income_aged_40_50"].sum() * 1.10,  # 10% increase for 40-50 age group
])

print(f"Original totals: {targets_matrix.sum().values}")
print(f"Target totals: {targets}")

Original totals: [661698.82562397 703699.75644773]
Target totals: [727868.70818636 774069.7320925 ]


In [3]:
# Initialize the Calibration object
calibrator = Calibration(
    data=data,
    weights=weights, 
    targets=targets
)

# Perform the calibration
calibrator.calibrate()

print(f"Original dataset size: {len(data)}")
print(f"Calibrated dataset size: {len(calibrator.data)}")
print(f"Number of calibrated weights: {len(calibrator.weights)}")

Reweighting progress: 100%|██████████| 32/32 [00:00<00:00, 56.04epoch/s, loss=501, count_observations=100, weights_mean=5.66, weights_std=3.04, weights_min=0.978]

Original dataset size: 100
Calibrated dataset size: 100
Number of calibrated weights: 100


In [6]:
# Verify the calibration results
calibrated_matrix = pd.DataFrame({
    "income_aged_20_30": ((calibrator.data["age"] >= 20) & (calibrator.data["age"] <= 30)).astype(float) * calibrator.data["income"],
    "income_aged_40_50": ((calibrator.data["age"] >= 40) & (calibrator.data["age"] <= 50)).astype(float) * calibrator.data["income"],
})

# Calculate final weighted totals
final_totals = calibrated_matrix.mul(calibrator.weights, axis=0).sum().values

print(f"Target totals: {targets}")
print(f"Final calibrated totals: {final_totals}")
print(f"Difference: {final_totals - targets}")
print(f"Relative error: {(final_totals - targets) / targets * 100}")

Target totals: [727868.70818636 774069.7320925 ]
Final calibrated totals: [5192262.89410487 4316896.38552935]
Difference: [4464394.1859185  3542826.65343685]
Relative error: [613.35157504 457.68830721]
